<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
#!/usr/bin/env python
# coding: utf-8

# <h1>Table of Contents<span class="tocSkip"></span></h1>
# <div class="toc"><ul class="toc-item"></ul></div>

import sys
import torch, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM  # === (изменено) убран pipeline, чтобы гарантированно не вызывался

import pandas as pd
from IPython.display import display


import math
from functools import reduce

import time as tm
t1_itogo = tm.time()

pd.set_option("display.max_rows", None) #убрали ограничение на число строк при отображении датафрейма
# ограничение на 20 строк
# pd.set_option("display.max_rows", 20)
from pathlib import Path




# убрала все предупреждения, потому что не смоогла вырубить предупреждения для модификации summary для коротких промптов
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# === BUILD STAMP (изменено) ===
print("BUILD: Qwen3 switch, SUMMARY hard-off | 2025-08-06")
try:
    import transformers, sys
    print("versions:", "torch=", torch.__version__, "transformers=", transformers.__version__, "python=", sys.version.split()[0])
except Exception as _e:
    print("version print failed:", _e)

if 'pipeline' in globals():
    print("⚠️ WARNING: found 'pipeline' symbol in globals(); ensure no 'from transformers import pipeline' left.")

"""
Скрипт сравнения распределений P(next_token) с поддержкой:
----------------------------------------------------------------
1. **Внутренних промптов** (список в коде) **или**
2. **Промптов из текстового файла** в той же директории.

Флаг `USE_FILE_PROMPTS` выбирает источник.
Если `True`, файл `PROMPT_FILE` читается целиком и используется как **один** длинный prompt.

Модификация «summary + слова» печатает краткое summary исходного prompt‑а.
Добавлена **безопасная токенизация**: если длиннее лимита модели (1024 для GPT‑2),
текст автоматически усечётся, а в консоль выведется предупреждение.
""";

C:\Users\cab\anaconda3\envs\torch241\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BUILD: Qwen3 switch, SUMMARY hard-off | 2025-08-06
versions: torch= 2.6.0+cpu transformers= 4.55.0 python= 3.10.18


In [2]:
t1_temp = tm.time()
# ---------- ПАРАМЕТРЫ ----------
# === QWEN3 SWITCH  ===
MODEL_NAME   = "Qwen/Qwen3-0.6B-Base"  # публичный идентификатор на HF; базовая (не чат) версия  # <-- укажите точный ID вашей Qwen3 модели, если другой (напр., локальное имя/путь)
TRUST_REMOTE_CODE = True      # для семейств Qwen обычно требуется
STEPS        = 80
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"

# Флаг, управляющий включением суммаризации как отдельной модификации.
# === QWEN3 SWITCH (изменено) === — теперь реально управляет созданием пайплайна
# === (изменено пользователем) — summary временно отключён
SUMMARY_ENABLED = False

# Суммаризация
SUMM_MODEL   = "sshleifer/distilbart-cnn-12-6"
CUSTOM_WORDS = "Сводка:"

# Выбор источника промптов
PROMPT_FILE      = "prompt.txt"


In [3]:
t1_temp = tm.time()

# ---------- 1. Модели ----------
# (изменено) — для Qwen добавлен trust_remote_code
_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=TRUST_REMOTE_CODE)
# (изменено) — для Qwen добавлен trust_remote_code
_model     = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=TRUST_REMOTE_CODE).to(DEVICE).eval()

# === DISABLE SUMMARY HARD (изменено) ===
# Полностью отключаем создание summarizer: никакого pipeline() вообще
_summarizer = None
if SUMMARY_ENABLED:
    print("⚠️ SUMMARY_ENABLED=True, но summarizer временно принудительно отключён."
          "Переключите SUMMARY_ENABLED=False или обновите torch>=2.6/выберите safetensors-модель.")
    SUMMARY_ENABLED = False

# (изменено) — более надёжное определение лимита контекста для разных архитектур
CTX_LIMIT = (
    getattr(_model.config, "max_position_embeddings", None)
    or getattr(_model.config, "n_positions", None)
    or getattr(_tokenizer, "model_max_length", 1024)
)
t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

при количестве шагов 10 этот блок занял 0 минут 6 секунд


In [4]:
t1_temp = tm.time()

# ---------- 2. Вспомогательные функции ----------
def _next_token_probs(ids):
    with torch.no_grad():
        logits = _model(ids).logits[0, -1]
    return torch.softmax(logits, dim=-1).cpu()

def _kl(p, q, eps=0):  # нормально работает при eps=1e-8
    p = p + eps
    q = q + eps
    return torch.sum(p * torch.log(p / q)).item()

def _cos(p, q):
    return F.cosine_similarity(p, q, dim=0).item()

def _multi_step(prompt: str, n: int):
    token_ids = _tokenizer.encode(prompt)
    if len(token_ids) > CTX_LIMIT:
        print(f"⚠️ Промпт содержит {len(token_ids)} токенов и будет усечён до {CTX_LIMIT}.")
    ids = _tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=CTX_LIMIT
    )["input_ids"].to(DEVICE)

    dists = []
    for _ in range(n):
        probs = _next_token_probs(ids)
        dists.append(probs)
        next_id = probs.argmax().unsqueeze(0).unsqueeze(0).to(DEVICE)
        ids = torch.cat([ids, next_id], dim=1)
    return dists

t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

при количестве шагов 10 этот блок занял 0 минут 0 секунд


In [5]:
t1_temp = tm.time()

# ---------- 3. Источник промптов и модификаций ----------

USE_FILE_PROMPTS = False           # или False
PROMPT_FILE = "prompt.txt"        # имя файла

if USE_FILE_PROMPTS:
    path = Path(PROMPT_FILE)  # 'prompt.txt' по-умолчанию
    if not path.is_file():
        raise FileNotFoundError(f"Файл {PROMPT_FILE} не найден")

    # читаем файл целиком → одна строка
    PROMPTS = [path.read_text(encoding="utf-8").strip()]
else:
    PROMPTS = [
        "give an example that uses __call__ in Python",
       # "The future of artificial intelligence depends on",
    ]


# (изменено) — строим список модификаций без summary; добавим её условно ниже
MODS = [
    ("original", lambda p: p),

#    ("add salutation", lambda p: "Dear user, " + p),
 #   ("префикс ======", lambda p: "="*10 + p),
  #   ("fuck", lambda p: "Fuck! " + p),
 ("префикс вопрос", lambda p: "I have a question. " + p),
 ("префикс вопрос с ошибкой", lambda p: "I have a kvestion. " + p),
#    ("суффикс 10 лет?", lambda p: p + " in the next decade?"),
]

if SUMMARY_ENABLED and _summarizer is not None:
    MODS.append(
        ("summary", lambda p: CUSTOM_WORDS + _summarizer(p, max_length=60, min_length=10, do_sample=False)[0]['summary_text'])
    )
t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

при количестве шагов 10 этот блок занял 0 минут 0 секунд


In [6]:
PROMPTS


['give an example that uses __call__ in Python']

In [7]:
t1_temp = tm.time()

title = "Печатаем для каждого промпта сам промпт и все его модификации"

print("\n" + "=" * 80)                       # верхняя рамка
print(title.center(80))                     # центрируем на 80 символов
print("=" * 80 + "\n")     
# печатаем все ориганилы промптов и для них все их модификации, промпт за промптом
for i, base_prompt in enumerate(PROMPTS, 1):
    # 🔹 Печатаем исходный текст промпта (можно обрезать при необходимости)
    print(f"🔹 PROMPT #{i}")
    print(f"📝 {base_prompt}")
    print()

    # 🔹 Перебираем *все* модификации, кроме 'original'
    for mod_name, mod_fn in MODS:
        if mod_name == "original":
            continue  # исходник уже выше

        mod_text = mod_fn(base_prompt)

        # Убираем служебный префикс, если это summary
        if mod_name == "summary":
            mod_text = mod_text[len(CUSTOM_WORDS):]

        # Красивый вывод модификации
        print(f"   ➡️ [{mod_name}] {mod_text}")



t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')


         Печатаем для каждого промпта сам промпт и все его модификации          

🔹 PROMPT #1
📝 give an example that uses __call__ in Python

   ➡️ [префикс вопрос] I have a question. give an example that uses __call__ in Python
   ➡️ [префикс вопрос с ошибкой] I have a kvestion. give an example that uses __call__ in Python
при количестве шагов 10 этот блок занял 0 минут 0 секунд


In [8]:
t1_temp = tm.time()
# === 7.5: единый кэш распределений для (prompt, mod) ===
dist_cache = {}  # ключ: (prompt, mod_name) -> список из STEPS тензоров Prob
for prompt in PROMPTS:
    for mod_name, mod_fn in MODS:
        mod_prompt = mod_fn(prompt)
        dist_cache[(prompt, mod_name)] = _multi_step(mod_prompt, STEPS)

t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

при количестве шагов 10 этот блок занял 0 минут 13 секунд


In [9]:
t1_temp = tm.time()



TOP_K_SHOW = 5  # ТОП-5 на каждом шаге

def _topk_rows_from_dists(dists, tokenizer, mod_name, top_k=TOP_K_SHOW):
    """
    Из уже посчитанных dists собирает строки:
    (Step, Rank, f"{mod_name} Token", f"{mod_name} Prob")
    """
    records = []
    for step_idx, probs in enumerate(dists, start=1):
        # если приходят логиты — раскомментировать:
        # probs = torch.softmax(probs, dim=-1)

        topk = torch.topk(probs, top_k)
        ids  = topk.indices.tolist()
        ps   = topk.values.tolist()

        # ранги по убыванию вероятности: 1..top_k
        for rank, (tok_id, p) in enumerate(zip(ids, ps), start=1):
            records.append({
                "Step": step_idx,
                "Rank": rank,
                f"{mod_name} Token": tokenizer.decode([int(tok_id)]),
                f"{mod_name} Prob":  float(p),
            })
    return pd.DataFrame.from_records(records)


def build_wide_table_for_prompt_from_cache(prompt, mods, tokenizer, dist_cache, top_k=TOP_K_SHOW):
    from functools import reduce
    per_mod_tables = []
    for mod_name, _ in mods:
        dists = dist_cache[(prompt, mod_name)]
        df_mod = _topk_rows_from_dists(dists, tokenizer, mod_name, top_k=top_k)
        per_mod_tables.append(df_mod)

    df_wide = reduce(
        lambda left, right: pd.merge(left, right, on=["Step", "Rank"], how="inner"),
        per_mod_tables
    )

    cols = ["Step", "Rank"]
    for mod_name, _ in mods:
        cols += [f"{mod_name} Token", f"{mod_name} Prob"]
    df_wide = df_wide.loc[:, cols].sort_values(["Step","Rank"]).reset_index(drop=True)
    return prompt, df_wide


def build_wide_table_for_prompt(prompt, mods, steps, tokenizer):
    """
    prompt: строка исходного промпта
    mods:   список пар (mod_name, mod_fn)
    steps:  число шагов (STEPS)
    tokenizer: ваш токенайзер
    ---
    Возвращает: (title_str, df_wide)
    где title_str — заголовок (полный промпт)
         df_wide  — «широкая» таблица, строки: Step, Rank; колонки: по модификациям (Token/Prob).
    """
    # 1) для каждой модификации один раз считаем dists и строим "узкую" таблицу
    per_mod_tables = []
    for mod_name, mod_fn in mods:
        mod_prompt = mod_fn(prompt)

        # ✅ считаем ОДИН раз
        dists = _multi_step(mod_prompt, steps)

        # превращаем в таблицу только top-K для печати
        df_mod = _topk_rows_from_dists(dists, tokenizer, mod_name, top_k=TOP_K_SHOW)
        per_mod_tables.append(df_mod)

    # 2) мерджим по ключам (Step, Rank), получаем «широкую» таблицу
    df_wide = reduce(
        lambda left, right: pd.merge(left, right, on=["Step", "Rank"], how="inner"),
        per_mod_tables
    )

    # 3) упорядочим колонки: Step, Rank, затем по модам (Token, Prob)
    cols = ["Step", "Rank"]
    for mod_name, _ in mods:
        cols += [f"{mod_name} Token", f"{mod_name} Prob"]
    df_wide = df_wide.loc[:, cols].sort_values(by=["Step", "Rank"], ascending=[True, True]).reset_index(drop=True)

    return prompt, df_wide


# ===== пример использования =====
# PROMPTS: список строк
# MODS:    [(mod_name, mod_fn), ...]
# STEPS:   число шагов
# _tokenizer: ваш токенайзер

# выбираем один промпт (или гоняем в цикле по всем)
for prompt in PROMPTS:
    title, df_wide =  build_wide_table_for_prompt_from_cache(
        prompt, MODS, _tokenizer, dist_cache, top_k=TOP_K_SHOW
    )

t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

при количестве шагов 10 этот блок занял 0 минут 0 секунд


In [10]:
t1_temp = tm.time()



# ---------- 1) Преобразуем «узкую» сводную таблицу к многоуровневым колонкам ----------
def prettify_wide_table(df_wide, mods, prob_fmt="{:.6f}", to_percent=False):
    """
    df_wide: таблица вида [Step, Rank, <mod1> Token, <mod1> Prob, <mod2> Token, <mod2> Prob, ...]
    mods:    список [(mod_name, mod_fn), ...] — порядок колонок возьмём ровно по этому списку
    """
    # 1) убедимся, что порядок колонок строго по mods
    cols = ["Step", "Rank"]
    for mod_name, _ in mods:
        cols += [f"{mod_name} Token", f"{mod_name} Prob"]
    df = df_wide.loc[:, cols].copy()

    # 2) делаем многоуровневые КОЛОНКИ: верхний уровень "Mod", средний — имя модификации, нижний — Token/Prob
    new_cols = []
    for c in df.columns:
        if c in ("Step", "Rank"):
            # это уйдёт в ИНДЕКС (см. ниже)
            continue
        # парсим имя "<mod> Token"/"<mod> Prob"
        mod_name, sub = c.rsplit(" ", 1)  # ["<mod>", "Token"/"Prob"]
        new_cols.append(("Mod", mod_name, sub))
    df_cols = pd.MultiIndex.from_tuples(new_cols, names=["", "Mod", "Metric"])

    # 3) переносим Step/Rank в индекс
    df_idxed = df.set_index(["Step", "Rank"])
    df_idxed.columns = df_cols
    df_idxed.index.names = ["Metric", ""]  # чтобы сверху слева было "Metric", а под ним уровни Step/Rank

    # 4) форматируем Prob (в проценты по желанию)
    if to_percent:
        def _fmt_prob(x):
            return "{:.2f}%".format(100.0 * x)
    else:
        def _fmt_prob(x):
            try:
                return prob_fmt.format(float(x))
            except Exception:
                return x

    # применим формат только к нижнему уровню "Prob"
    prob_cols = [c for c in df_idxed.columns if c[-1] == "Prob"]

    # 5) Styler: зебра, липкий индекс, выравнивания, шапки
    styler = (
        df_idxed.style
        .format({col: _fmt_prob for col in prob_cols})
        .set_table_styles([
            # фиксируем первый столбец индекса (Step) и второй (Rank) — липкие слева
            {'selector': 'th.row_heading.level0', 'props': 'position: sticky; left: 0; z-index: 3; background: white;'},
            {'selector': 'th.row_heading.level1', 'props': 'position: sticky; left: 60px; z-index: 3; background: white;'},  # подстрой left при необходимости
            # фиксируем шапку
            {'selector': 'th.col_heading', 'props': 'position: sticky; top: 0; z-index: 2; background: #f7f7f7;'},
            # общий стиль таблицы
            {'selector': 'table', 'props': 'border-collapse: collapse; font-family: system-ui, sans-serif;'},
            {'selector': 'th, td', 'props': 'padding: 6px 10px; border: 1px solid #ddd;'},
            # верхний уровень "Mod" — покрупнее и по центру
            {'selector': 'th.col_heading.level0', 'props': 'text-align: center; font-weight: 700;'},
            {'selector': 'th.col_heading.level1', 'props': 'text-align: center;'},
            {'selector': 'th.col_heading.level2', 'props': 'text-align: center;'},
            # заголовки индекса слева
            {'selector': 'th.row_heading', 'props': 'text-align: right;'},
        ])
        .set_properties(subset=[c for c in df_idxed.columns if c[-1] == "Token"], **{'text-align': 'left',  'white-space': 'pre'})
        .set_properties(subset=[c for c in df_idxed.columns if c[-1] == "Prob"],  **{'text-align': 'right'})
        .apply(lambda s: ['background-color: #fafafa' if (i % 2) else '' for i in range(len(s))], axis=0)  # зебра по строкам
    )

    return styler


# ---------- 2) Печать «PROMPT: …» + красивая таблица ----------
def display_prompt_table(prompt_text, df_wide, mods, *, percent=False, prob_fmt="{:.6f}"):
    print("🔹 PROMPT:")
    # чтобы переносы/пробелы в токенах/заголовке отображались предсказуемо
    print(prompt_text)
    styler = prettify_wide_table(df_wide, mods, prob_fmt=prob_fmt, to_percent=percent)
    return styler

t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

при количестве шагов 10 этот блок занял 0 минут 0 секунд


In [11]:
t1_temp = tm.time()



def _slice_steps(df_wide, *, start_step=None, end_step=None, head_first=None, tail_last=None):
    """
    Возвращает срез df_wide по шагам.

    Варианты:
      1) Диапазон: start_step..end_step (оба включительно).
      2) Голова+хвост: head_first первых шагов И/ИЛИ tail_last последних шагов.
         - Если заданы оба (head_first и tail_last), берётся объединение этих шагов.
      Примечание: Диапазон (start_step/end_step) и голова/хвост взаимоисключающие.
    """
    # нормализуем вход
    if (start_step is not None or end_step is not None) and (head_first is not None or tail_last is not None):
        raise ValueError("Выбери либо диапазон (start/end), либо голову/хвост (head_first/tail_last), но не оба варианта одновременно.")

    # базовый порядок шагов
    steps = sorted(df_wide["Step"].unique())

    # 1) Диапазон
    if (start_step is not None) or (end_step is not None):
        lo = start_step if start_step is not None else steps[0]
        hi = end_step   if end_step   is not None else steps[-1]
        if lo > hi:
            lo, hi = hi, lo
        keep = [s for s in steps if lo <= s <= hi]
        return df_wide[df_wide["Step"].isin(keep)].copy()

    # 2) Голова+хвост
    keep_set = set()
    if head_first is not None and head_first > 0:
        keep_set.update(steps[:head_first])
    if tail_last is not None and tail_last > 0:
        keep_set.update(steps[-tail_last:])

    if keep_set:
        keep = [s for s in steps if s in keep_set]
        return df_wide[df_wide["Step"].isin(keep)].copy()

    # без ограничений — возвращаем как есть
    return df_wide.copy()


# === обновление: теперь display_prompt_table умеет срезы ===

def display_prompt_table(prompt_text, df_wide, mods, *,
                         percent=False, prob_fmt="{:.6f}",
                         start_step=None, end_step=None,
                         head_first=None, tail_last=None):
    """
    Печатает PROMPT и красивую таблицу.
    Доп.параметры:
      - start_step, end_step: диапазон шагов (включительно)
      - head_first, tail_last: вывести a первых и b последних шагов
    """
    print("🔹 PROMPT:")
    print(prompt_text)

    # применяем срез шагов
    df_sliced = _slice_steps(
        df_wide,
        start_step=start_step,
        end_step=end_step,
        head_first=head_first,
        tail_last=tail_last
    )

    # важно заново отсортировать по Step/Rank после среза
    df_sliced = df_sliced.sort_values(["Step", "Rank"]).reset_index(drop=True)

    # далее — как раньше
    styler = prettify_wide_table(df_sliced, mods, prob_fmt=prob_fmt, to_percent=percent)
    return styler

t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')


при количестве шагов 10 этот блок занял 0 минут 0 секунд


In [12]:
# 1) Диапазон шагов: с 10 по 25 включительно
#styler = display_prompt_table(title, df_wide, MODS, start_step=10, end_step=25)
#styler

# 2) Голова+хвост: 5 первых и 2 последних шага
styler = display_prompt_table(title, df_wide, MODS, head_first=5, tail_last=2)
styler

# 3) Только 8 первых шагов
#styler = display_prompt_table(title, df_wide, MODS, head_first=8)
#styler

# 4) Только 3 последних шага
#styler = display_prompt_table(title, df_wide, MODS, tail_last=3)
#styler


🔹 PROMPT:
give an example that uses __call__ in Python


In [13]:


def best_continuation_from_dists(dists, tokenizer, top_rank=1):
    """
    dists: Iterable[Tensor] — распределения по шагам (каждый Tensor — probs по vocab)
    tokenizer: ваш токенайзер
    top_rank: какой ранг брать (1 = самый вероятный токен)
    ---
    Возвращает:
      best_text: str
      token_ids: List[int]
      logprob_sum: float  (сумма лог-вероятностей выбранных токенов)
    """
    token_ids = []
    logprob_sum = 0.0

    for probs in dists:
        # берём top-k (нам нужен top_rank-ый)
        k = max(top_rank, 1)
        topk = torch.topk(probs, k)
        idx = topk.indices[top_rank - 1].item()
        p   = topk.values [top_rank - 1].item()
        token_ids.append(idx)
        # защитимся от p = 0 (должно быть >0, но на всякий случай)
        logprob_sum += math.log(max(p, 1e-45))

    # ВАЖНО: декодим ids, чтобы корректно восстановить пробелы/пунктуацию
    best_text = tokenizer.decode(token_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return best_text, token_ids, logprob_sum

t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

при количестве шагов 10 этот блок занял 0 минут 0 секунд


In [14]:
t1_temp = tm.time()

# === 12: лучший текст из уже посчитанных dists (без _multi_step) ===
all_rows_12 = []

for prompt in PROMPTS:
    for (mod_name, _mod_fn) in MODS:
        dists = dist_cache[(prompt, mod_name)]              # <-- берём из кэша
        # 12.1 собрать «лучшее» продолжение (rank-1 по каждому шагу)
        best_text, token_ids, logprob_sum = best_continuation_from_dists(
            dists, _tokenizer, top_rank=1
        )

        # 12.2 (опц.) если нужен табличный top-K по шагам — тоже из dists
        df_local = _topk_rows_from_dists(dists, _tokenizer, mod_name, top_k=TOP_K_SHOW)
        df_local["PromptFull"] = prompt
        df_local["Prompt"]     = prompt[:120] + ("…" if len(prompt) > 120 else "")
        df_local["Mod"]        = mod_name
        all_rows_12.append(df_local)

        # 12.3 печать «лучшего» текста (или сохрани как тебе нужно)
        print(f"[{mod_name}] score={logprob_sum:.2f} :: {best_text}")

df_probs_12 = pd.concat(all_rows_12, ignore_index=True)

t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

[original] score=-4.85 :: 
Sure! Here's an example that uses the
[префикс вопрос] score=-5.45 :: 
Sure! Here's an example that uses the
[префикс вопрос с ошибкой] score=-16.89 :: . I have a class that has a method that
при количестве шагов 10 этот блок занял 0 минут 0 секунд


In [15]:
t1_temp = tm.time()
# === 13: KL и Cos между модификациями — тоже из кэша ===
TRUNC = 60
dist_records = []

for prompt in PROMPTS:
    base_dists = dist_cache[(prompt, "original")]          # базовая модификация

    for (mod_name, _mod_fn) in MODS:
        if mod_name == "original":
            continue

        mod_dists = dist_cache[(prompt, mod_name)]         # обе последовательности уже есть

        for step_idx, (p_base, p_mod) in enumerate(zip(base_dists, mod_dists), start=1):
            dist_records.append({
                "Prompt": prompt[:TRUNC] + ('…' if len(prompt) > TRUNC else ''),
                "Step":   step_idx,
                "Mod":    mod_name,
                "KL":     _kl (p_base, p_mod, eps=1e-8),
                "Cos":    _cos(p_base, p_mod),
            })

dist_df = pd.DataFrame(dist_records)

t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

при количестве шагов 10 этот блок занял 0 минут 0 секунд


In [16]:
# In[13]:
t1_temp = tm.time()

# (1) глобальная настройка вывода -----------------
pd.set_option("display.max_columns", None)   # показывать все колонки
# ----- 1. Cosine ------------------------------------------------
cos_df = dist_df.pivot_table(
    index   = ["Prompt", "Step"],     # мульти‑индекс: промпт → шаг
    columns = "Mod",                  # колонки — модификации
    values  = "Cos"                   # берём только Cos
).sort_index()


t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')


при количестве шагов 10 этот блок занял 0 минут 0 секунд


In [17]:
t1_temp = tm.time()

# ----- 2. KL ----------------------------------------------------
kl_df = dist_df.pivot_table(
    index   = ["Prompt", "Step"],
    columns = "Mod",
    values  = "KL"
).sort_index()

t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

при количестве шагов 10 этот блок занял 0 минут 0 секунд


In [18]:
# ----- 3. Печать ------------------------------------------------
t1_temp = tm.time()

print("◾️ Cosine similarity")
display(cos_df)

print("\n◾️ KL divergence")
display(kl_df)


# In[14]:

t2_temp = tm.time()
print("при количестве шагов", STEPS ,'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

◾️ Cosine similarity


Mod                                                префикс вопрос  \
Prompt                                       Step                   
give an example that uses __call__ in Python 1           0.773793   
                                             2           0.972140   
                                             3           0.976194   
                                             4           0.980585   
                                             5           0.999854   
                                             6           0.999840   
                                             7           1.000000   
                                             8           0.981274   
                                             9           0.991467   
                                             10          0.999925   

Mod                                                префикс вопрос с ошибкой  
Prompt                                       Step                            
give an example that uses __call__ in Python 1                     0.651662  
                                             2                     0.001621  
                                             3                     0.000500  
                                             4                     0.000424  
                                             5                     0.000032  
                                             6                     0.001387  
                                             7                     0.000304  
                                             8                     0.003149  
                                             9                     0.000010  
                                             10                    0.010085


◾️ KL divergence


Mod                                                префикс вопрос  \
Prompt                                       Step                   
give an example that uses __call__ in Python 1           0.298924   
                                             2           0.181199   
                                             3           0.035823   
                                             4           0.047695   
                                             5           0.003874   
                                             6           0.005809   
                                             7           0.000022   
                                             8           0.027870   
                                             9           0.016694   
                                             10          0.000863   

Mod                                                префикс вопрос с ошибкой  
Prompt                                       Step                            
give an example that uses __call__ in Python 1                     0.485331  
                                             2                    10.568040  
                                             3                    10.784346  
                                             4                    11.340172  
                                             5                    12.582747  
                                             6                     8.051434  
                                             7                     9.372339  
                                             8                     6.646872  
                                             9                    12.822200  
                                             10                    6.048084

при количестве шагов 10 этот блок занял 0 минут 0 секунд


In [19]:

t2_itogo = tm.time()
print("при количестве шагов", STEPS ,'весь код занял', round(t2_itogo - t1_itogo)//60,'минут', 
      round(t2_itogo - t1_itogo)%60,'секунд')


при количестве шагов 10 весь код занял 0 минут 20 секунд
